# Power Flow Analysis and SLD Generator

This notebook allows you to select and run power flow analysis for different network topologies. The Single Line Diagrams (SLDs) will be generated and saved in the 'SLD diagram' folder.

## Overview

This repository contains five different network topologies for power flow analysis:
1. Transformer Station Near Aalamoen (22KV)
2. Transformer Station Near Aalamoen (33KV)
3. Transformer Station Near Aalamoen with BESS (33KV)
4. Transformer Station Near Solar Plant (22KV)
5. Transformer Station Near Solar Plant (33KV)

Each topology analyzes different voltage levels and configurations for optimal power transmission.

In [ ]:
# Import required libraries
import os
import sys
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Create SLD diagram directory if it doesn't exist
os.makedirs('SLD diagram', exist_ok=True)

# Check for required packages
try:
    import pypowsybl
except ImportError:
    print("⚠️ Warning: pypowsybl package is not installed. Install it with: pip install pypowsybl")

try:
    import nbformat
except ImportError:
    print("⚠️ Warning: nbformat package is not installed. Install it with: pip install nbformat")

## Available Network Topologies

Select one of the following network topologies to analyze. Each topology file is located in the CSS folder.

In [ ]:
# Define the available topology notebooks
topology_notebooks = [
    {
        "name": "Transformer Station Near Aalamoen (22KV)",
        "path": "CSS/SLD_Transformer_Station_Near_Aalamoen_22KV.ipynb",
        "description": "Analysis of transformer station near Aalamoen with 22KV collection system"
    },
    {
        "name": "Transformer Station Near Aalamoen (33KV)",
        "path": "CSS/SLD_Transformer_Station_Near_Aalamoen_33KV.ipynb",
        "description": "Analysis of transformer station near Aalamoen with 33KV collection system"
    },
    {
        "name": "Transformer Station Near Aalamoen with BESS (33KV)",
        "path": "CSS/SLD_Transformer_Station_Near_Aalamoen_33KV_with_BESS_extra_bus_in_solar_plant.ipynb",
        "description": "Analysis of transformer station near Aalamoen with 33KV collection system and Battery Energy Storage System"
    },
    {
        "name": "Transformer Station Near Solar Plant (22KV)",
        "path": "CSS/SLD_Transformer_Station_Near_the_Solar_Plant_22KV.ipynb",
        "description": "Analysis of transformer station near the solar plant with 22KV collection system"
    },
    {
        "name": "Transformer Station Near Solar Plant (33KV)",
        "path": "CSS/SLD_Transformer_Station_Near_the_Solar_Plant_33KV.ipynb",
        "description": "Analysis of transformer station near the solar plant with 33KV collection system"
    }
]

# Check if all topology notebooks exist
missing_notebooks = []
for topo in topology_notebooks:
    if not os.path.exists(topo["path"]):
        missing_notebooks.append(topo["name"])

if missing_notebooks:
    print("⚠️ Warning: The following topology notebooks were not found:")
    for nb in missing_notebooks:
        print(f"  - {nb}")
    print("\nPlease make sure these notebooks are placed in the CSS folder.")
else:
    print("✅ All topology notebooks found in the CSS folder.")

In [ ]:
# Create a dropdown widget to select the topology
dropdown = widgets.Dropdown(
    options=[(t["name"], i) for i, t in enumerate(topology_notebooks)],
    description='Select Topology:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Display the dropdown
display(dropdown)

# Show description of selected topology
description_output = widgets.Output()
def show_description(change):
    with description_output:
        description_output.clear_output()
        print(topology_notebooks[change['new']]['description'])
dropdown.observe(show_description, names='value')
display(description_output)

# Initial description display
with description_output:
    print(topology_notebooks[0]['description'])

## Run Selected Topology

Click the button below to run the selected topology. The analysis will be performed and SLD diagrams will be generated in the 'SLD diagram' folder.

**Note**: Running a topology may take several minutes depending on the complexity.

In [ ]:
# Function to modify a notebook to save outputs to SLD diagram folder
def prepare_notebook(notebook_path):
    # Don't modify if notebook doesn't exist
    if not os.path.exists(notebook_path):
        return False
    
    # Read the notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb_content = f.read()
    
    # Check if the notebook already imports os
    if "import os" not in nb_content:
        print("Adding os import to notebook...")
        # We'll add this when executing via %run
    
    return True

# Create a button to run the selected topology
run_button = widgets.Button(
    description='Run Analysis',
    button_style='success',
    tooltip='Click to run the selected topology analysis',
    icon='play'
)

# Create output widget to display results
output = widgets.Output()

# Function to run when the button is clicked
def run_topology(b):
    with output:
        output.clear_output()
        selected_idx = dropdown.value
        selected_topology = topology_notebooks[selected_idx]
        print(f"Running: {selected_topology['name']}")
        print(f"Loading notebook: {selected_topology['path']}")
        
        # Check if the notebook exists
        if not os.path.exists(selected_topology['path']):
            print(f"\nError: Notebook file not found at {selected_topology['path']}")
            print("Please make sure the notebook exists in the CSS folder.")
            return
        
        # Prepare the notebook
        prepare_notebook(selected_topology['path'])
        
        # Create the output directory
        os.makedirs('SLD diagram', exist_ok=True)
        print("\nOutput SLD diagrams will be saved to: ./SLD diagram/")
        
        # Import os in case the notebook doesn't
        print("\nRunning notebook... (this may take a few minutes)")
        try:
            # Import os in the current context
            import os
            os.makedirs('SLD diagram', exist_ok=True)
            
            # First run a setup cell to ensure SLD diagram folder exists
            setup_cell = """
import os
os.makedirs('SLD diagram', exist_ok=True)
# Monkey patch the write_matrix_multi_substation_single_line_diagram_svg function
original_write_svg = None
try:
    import pypowsybl.network as pn
    original_write_svg = pn.Network.write_matrix_multi_substation_single_line_diagram_svg
    def patched_write_svg(self, layout, svg_file, parameters):
        # Call original first
        original_write_svg(self, layout, svg_file, parameters)
        # Then save to SLD diagram folder
        filename = os.path.basename(svg_file)
        sld_path = os.path.join('SLD diagram', filename)
        original_write_svg(self, layout, sld_path, parameters)
        print(f"SLD diagram saved to: {sld_path}")
    pn.Network.write_matrix_multi_substation_single_line_diagram_svg = patched_write_svg
except (ImportError, AttributeError) as e:
    print(f"Could not patch SVG writing function: {e}")
            """
            exec(setup_cell)
            
            # Now use the %run magic to execute the selected notebook
            %run $selected_topology['path']
            
            # Restore original function if we patched it
            if original_write_svg is not None:
                import pypowsybl.network as pn
                pn.Network.write_matrix_multi_substation_single_line_diagram_svg = original_write_svg
            
            print("\nAnalysis completed successfully!")
            print("SLD diagrams have been saved to the 'SLD diagram' folder.")
            
            # Refresh the diagram list
            refresh_diagrams(None)
            
        except Exception as e:
            print(f"\nError running topology: {str(e)}")
            import traceback
            traceback.print_exc()

# Register the button click event
run_button.on_click(run_topology)

# Display the button and output
display(run_button)
display(output)

## View Generated SLD Diagrams

After running a topology, you can browse the generated SLD diagrams from the list below.
Click the "Refresh Diagram List" button to update the list after running an analysis.

In [ ]:
def list_sld_diagrams():
    """List and create links to available SLD diagrams"""
    diagram_dir = 'SLD diagram'
    if not os.path.exists(diagram_dir):
        return HTML("<p>No SLD diagrams found. Run a topology analysis first.</p>")
    
    files = [f for f in os.listdir(diagram_dir) if f.endswith('.svg')]
    
    if not files:
        return HTML("<p>No SLD diagrams found. Run a topology analysis first.</p>")
    
    # Group files by topology
    file_groups = {}
    for f in files:
        # Extract topology name from filename
        if '_22KV' in f or '_22kv' in f:
            key = '22KV'
        elif '_33KV_with_BESS' in f or '_33kv_with_bess' in f or '_33KV_extraSG' in f:
            key = '33KV with BESS'
        elif '_33KV' in f or '_33kv' in f:
            key = '33KV'
        else:
            key = 'Other'
        
        if key not in file_groups:
            file_groups[key] = []
        file_groups[key].append(f)
    
    # Generate HTML with grouped files
    html = "<h3>Generated SLD Diagrams</h3>"
    
    # Display file count
    html += f"<p>Total SVG files: {len(files)}</p>"
    
    # List files by topology group
    for group, group_files in file_groups.items():
        html += f"<h4>{group} Topology: ({len(group_files)} files)</h4><ul>"
        for f in sorted(group_files):
            path = os.path.join(diagram_dir, f)
            html += f"<li><a href='{path}' target='_blank'>{f}</a></li>"
        html += "</ul>"
    
    return HTML(html)

# Button to refresh the diagram list
refresh_button = widgets.Button(
    description='Refresh Diagram List',
    tooltip='Click to refresh the list of available SLD diagrams',
    icon='refresh'
)

diagram_output = widgets.Output()

def refresh_diagrams(b):
    with diagram_output:
        diagram_output.clear_output()
        display(list_sld_diagrams())

refresh_button.on_click(refresh_diagrams)

display(refresh_button)
with diagram_output:
    display(list_sld_diagrams())

## Working with the Generated SLD Diagrams

The generated SLD (Single Line Diagram) files are SVG format, which can be viewed in any web browser or SVG-compatible application. To work with these files:

1. **View in Browser**: Click the links above to open the SVG files in a new browser tab
2. **Download**: Right-click on the links and select "Save Link As..." to download the files
3. **Edit**: Use vector graphics tools like Inkscape, Adobe Illustrator, or online SVG editors to modify the diagrams if needed

## Tips for Working with the Notebooks

If you want to modify any of the topology notebooks directly:

1. Open the CSS folder and select the desired notebook
2. Make your modifications to the network configuration, parameters, or analysis code
3. Remember that output SLD diagrams should be saved to the 'SLD diagram' folder using the following pattern:

```python
# Modified to save in SLD diagram folder:
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], ['S3', 'S4']], 
    os.path.join('SLD diagram', 'sld_2x2_matrix.svg'),
    parameters=param
)
```

## Advanced: Creating New Topology Notebooks

To create a new topology notebook:

1. Duplicate one of the existing notebooks in the CSS folder
2. Rename it to reflect your new topology
3. Modify the network configuration, parameters, and analysis code
4. Update the `topology_notebooks` list in this notebook to include your new topology

Remember to include proper documentation and save outputs to the 'SLD diagram' folder.